# Advanced Geometry Processing Tutorial

This notebook explores advanced geometry processing features in EasyCablePulling.

## Learning Objectives

- Understand geometry fitting algorithms
- Work with different primitive types
- Analyze fitting accuracy and errors
- Debug geometry processing issues
- Optimize fitting parameters

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from easycablepulling.core.models import CableSpec, DuctSpec, Section
from easycablepulling.geometry import GeometryProcessor
from easycablepulling.geometry.fitter import AdvancedFitter
from easycablepulling.io import load_route_from_dxf

plt.rcParams['figure.figsize'] = (14, 8)
print("Advanced geometry modules imported!")

## 1. Examine Different Route Geometries

Let's load and examine different types of route geometries to understand the fitting challenges.

In [ ]:
# Load different route types
routes = {
    "Straight": "../tests/data/straight_route.dxf",
    "S-Curve": "../tests/data/s_curve_route.dxf",
    "Complex": "../tests/data/complex_route.dxf",
    "Many Bends": "../tests/data/many_bends_route.dxf"
}

loaded_routes = {}

for name, file_path in routes.items():
    try:
        route = load_route_from_dxf(Path(file_path))
        loaded_routes[name] = route
        
        print(f"{name:12}: {len(route.sections)} sections, {route.total_length:.1f}m total")
        
        for section in route.sections:
            points = len(section.original_polyline)
            length = section.original_length
            print(f"              Section {section.id}: {points} points, {length:.1f}m")
            
    except Exception as e:
        print(f"{name:12}: Failed to load - {e}")

In [ ]:
# Visualize original route geometries
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, (name, route) in enumerate(loaded_routes.items()):
    if i >= 4:  # Only plot first 4
        break
        
    ax = axes[i]
    
    for section in route.sections:
        points = np.array(section.original_polyline)
        if len(points) > 0:
            ax.plot(points[:, 0]/1000, points[:, 1]/1000, 'b-', linewidth=2, marker='o', markersize=3)
            
            # Mark start and end
            ax.plot(points[0, 0]/1000, points[0, 1]/1000, 'go', markersize=8, label='Start')
            ax.plot(points[-1, 0]/1000, points[-1, 1]/1000, 'ro', markersize=8, label='End')
    
    ax.set_title(f"{name} Route Geometry")
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    if i == 0:
        ax.legend()

plt.tight_layout()
plt.show()

## 2. Geometry Fitting Process

Let's examine how the geometry fitting process works and analyze the results.

In [ ]:
# Test geometry processing with different tolerances
test_route = loaded_routes["S-Curve"]
test_section = test_route.sections[0]

tolerances = [0.5, 1.0, 2.0, 5.0]  # mm
fitting_results = {}

for tolerance in tolerances:
    processor = GeometryProcessor(
        tolerance_mm=tolerance,
        min_segment_length=50.0,
        max_iterations=100
    )
    
    # Process just the geometry
    result = processor.process_route(test_route)
    fitting_results[tolerance] = result
    
    if result.success:
        fitted_section = result.route.sections[0]
        primitives = len(fitted_section.primitives)
        error = result.fitting_results[0].total_error if result.fitting_results else 0
        
        print(f"Tolerance {tolerance:4.1f}mm: {primitives} primitives, {error:.3f}mm total error")
    else:
        print(f"Tolerance {tolerance:4.1f}mm: Fitting failed")

In [ ]:
# Visualize fitting results for different tolerances
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

original_points = np.array(test_section.original_polyline)

for i, (tolerance, fit_result) in enumerate(fitting_results.items()):
    if i >= 4:
        break
        
    ax = axes[i]
    
    # Plot original polyline
    ax.plot(original_points[:, 0]/1000, original_points[:, 1]/1000, 
           'b-', linewidth=2, alpha=0.7, label='Original')
    ax.plot(original_points[:, 0]/1000, original_points[:, 1]/1000, 
           'bo', markersize=4, alpha=0.5)
    
    if fit_result.success and len(fit_result.route.sections) > 0:
        fitted_section = fit_result.route.sections[0]
        
        # Plot fitted primitives
        for j, primitive in enumerate(fitted_section.primitives):
            if hasattr(primitive, 'start_point') and hasattr(primitive, 'end_point'):
                # Straight section
                start = np.array(primitive.start_point) / 1000
                end = np.array(primitive.end_point) / 1000
                ax.plot([start[0], end[0]], [start[1], end[1]], 
                       'r-', linewidth=4, alpha=0.8, label='Fitted' if j == 0 else '')
            elif hasattr(primitive, 'center_point') and hasattr(primitive, 'radius_m'):
                # Bend section - approximate with arc
                center = np.array(primitive.center_point) / 1000
                radius = primitive.radius_m / 1000
                
                # Create arc points for visualization
                angles = np.linspace(0, np.radians(abs(primitive.angle_deg)), 20)
                if hasattr(primitive, 'start_angle_deg'):
                    start_angle = np.radians(primitive.start_angle_deg)
                    angles += start_angle
                
                arc_x = center[0] + radius * np.cos(angles)
                arc_y = center[1] + radius * np.sin(angles)
                ax.plot(arc_x, arc_y, 'r-', linewidth=4, alpha=0.8, 
                       label='Fitted' if j == 0 else '')
        
        # Show fitting error
        if fit_result.fitting_results:
            total_error = fit_result.fitting_results[0].total_error
            ax.set_title(f"Tolerance {tolerance}mm\nTotal Error: {total_error:.2f}mm")
        else:
            ax.set_title(f"Tolerance {tolerance}mm")
    else:
        ax.set_title(f"Tolerance {tolerance}mm\nFitting Failed")
    
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    if i == 0:
        ax.legend()

plt.tight_layout()
plt.show()

## 3. Analyze Primitive Types

Let's examine what types of primitives are fitted to different route sections.

In [ ]:
# Analyze primitives in complex route
complex_route = loaded_routes.get("Complex")
if complex_route:
    processor = GeometryProcessor(tolerance_mm=2.0)
    complex_result = processor.process_route(complex_route)
    
    if complex_result.success:
        print("Complex Route Primitive Analysis:")
        
        for section in complex_result.route.sections:
            print(f"\nSection {section.id}:")
            
            for i, primitive in enumerate(section.primitives):
                prim_type = type(primitive).__name__
                length = primitive.length()
                
                print(f"  Primitive {i}: {prim_type}, {length:.1f}m")
                
                if hasattr(primitive, 'radius_m'):
                    print(f"    Radius: {primitive.radius_m:.1f}m")
                    print(f"    Angle: {primitive.angle_deg:.1f}°")
                    if hasattr(primitive, 'bend_type'):
                        print(f"    Bend type: {primitive.bend_type}")
    else:
        print(f"Complex route fitting failed: {complex_result.message}")
else:
    print("Complex route not available")

## 4. Fitting Accuracy Analysis

Let's analyze the accuracy of geometry fitting and understand the trade-offs.

In [ ]:
# Detailed fitting accuracy analysis
def analyze_fitting_accuracy(route, tolerance_range):
    """Analyze fitting accuracy across tolerance range."""
    
    results = []
    
    for tolerance in tolerance_range:
        processor = GeometryProcessor(tolerance_mm=tolerance)
        fit_result = processor.process_route(route)
        
        if fit_result.success and fit_result.fitting_results:
            fitting_result = fit_result.fitting_results[0]
            
            results.append({
                'tolerance': tolerance,
                'primitives': len(fitting_result.primitives),
                'total_error': fitting_result.total_error,
                'max_error': fitting_result.max_error,
                'success': True
            })
        else:
            results.append({
                'tolerance': tolerance,
                'primitives': 0,
                'total_error': float('inf'),
                'max_error': float('inf'),
                'success': False
            })
    
    return results

# Test on S-curve route
tolerance_range = np.logspace(-0.5, 1.5, 10)  # 0.3mm to 30mm
s_curve_route = loaded_routes.get("S-Curve")

if s_curve_route:
    accuracy_results = analyze_fitting_accuracy(s_curve_route, tolerance_range)
    
    # Plot accuracy vs complexity trade-off
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    successful_results = [r for r in accuracy_results if r['success']]
    
    if successful_results:
        tolerances = [r['tolerance'] for r in successful_results]
        total_errors = [r['total_error'] for r in successful_results]
        primitive_counts = [r['primitives'] for r in successful_results]
        
        # Plot 1: Error vs tolerance
        ax1.loglog(tolerances, total_errors, 'bo-', linewidth=2, markersize=6)
        ax1.set_xlabel('Tolerance (mm)')
        ax1.set_ylabel('Total Fitting Error (mm)')
        ax1.set_title('Fitting Accuracy vs Tolerance')
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Complexity vs tolerance
        ax2.semilogx(tolerances, primitive_counts, 'ro-', linewidth=2, markersize=6)
        ax2.set_xlabel('Tolerance (mm)')
        ax2.set_ylabel('Number of Primitives')
        ax2.set_title('Model Complexity vs Tolerance')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print optimal tolerance recommendation
        # Find tolerance that balances accuracy and complexity
        normalized_errors = np.array(total_errors) / max(total_errors)
        normalized_complexity = np.array(primitive_counts) / max(primitive_counts)
        
        # Combined score (lower is better)
        scores = normalized_errors + 0.3 * normalized_complexity
        optimal_idx = np.argmin(scores)
        
        optimal_tolerance = tolerances[optimal_idx]
        print(f"\nRecommended tolerance: {optimal_tolerance:.1f}mm")
        print(f"  Primitives: {primitive_counts[optimal_idx]}")
        print(f"  Total error: {total_errors[optimal_idx]:.2f}mm")
    
else:
    print("S-Curve route not available for accuracy analysis")

## 5. Route Splitting Analysis

Let's examine how route splitting works and its effects on the analysis.

In [ ]:
# Test route splitting with different cable lengths
complex_route = loaded_routes.get("Complex")
if complex_route:
    
    cable_lengths = [200.0, 500.0, 1000.0, 2000.0]  # meters
    splitting_results = {}
    
    for max_length in cable_lengths:
        processor = GeometryProcessor(tolerance_mm=2.0)
        
        result = processor.process_route(
            complex_route,
            enable_splitting=True,
            max_cable_length=max_length
        )
        
        splitting_results[max_length] = result
        
        if result.success:
            original_sections = len(complex_route.sections)
            final_sections = len(result.route.sections)
            sections_added = result.splitting_result.sections_created if result.splitting_result else 0
            
            print(f"Max length {max_length:6.0f}m: {original_sections} → {final_sections} sections (+{sections_added})")
        else:
            print(f"Max length {max_length:6.0f}m: Processing failed")
else:
    print("Complex route not available for splitting analysis")

In [ ]:
# Visualize splitting results
if complex_route and splitting_results:
    
    # Choose two different splitting results to compare
    comparison_lengths = [500.0, 200.0]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    for i, max_length in enumerate(comparison_lengths):
        if max_length in splitting_results:
            split_result = splitting_results[max_length]
            ax = axes[i]
            
            if split_result.success:
                # Plot original route
                for section in complex_route.sections:
                    points = np.array(section.original_polyline)
                    ax.plot(points[:, 0]/1000, points[:, 1]/1000, 
                           'b-', linewidth=2, alpha=0.5, label='Original' if i == 0 else '')
                
                # Plot split sections with different colors
                colors = plt.cm.Set3(np.linspace(0, 1, len(split_result.route.sections)))
                
                for j, section in enumerate(split_result.route.sections):
                    for primitive in section.primitives:
                        if hasattr(primitive, 'start_point') and hasattr(primitive, 'end_point'):
                            start = np.array(primitive.start_point) / 1000
                            end = np.array(primitive.end_point) / 1000
                            ax.plot([start[0], end[0]], [start[1], end[1]], 
                                   color=colors[j], linewidth=4, alpha=0.8,
                                   label=f'Section {j+1}' if len(split_result.route.sections) <= 5 else '')
                
                sections_count = len(split_result.route.sections)
                ax.set_title(f"Max Cable Length: {max_length}m\n{sections_count} sections")
            else:
                ax.set_title(f"Max Cable Length: {max_length}m\nProcessing Failed")
            
            ax.set_xlabel('X (km)')
            ax.set_ylabel('Y (km)')
            ax.grid(True, alpha=0.3)
            ax.axis('equal')
            if i == 0:
                ax.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot visualize splitting results")

## 6. Custom Geometry Processing

Let's explore custom geometry processing parameters and their effects.

In [ ]:
# Test different processing parameters
many_bends_route = loaded_routes.get("Many Bends")
if many_bends_route:
    
    processing_configs = {
        "Conservative": {
            "tolerance_mm": 0.5,
            "min_segment_length": 100.0,
            "max_iterations": 200
        },
        "Balanced": {
            "tolerance_mm": 2.0,
            "min_segment_length": 50.0,
            "max_iterations": 100
        },
        "Fast": {
            "tolerance_mm": 5.0,
            "min_segment_length": 25.0,
            "max_iterations": 50
        }
    }
    
    print("Processing Configuration Comparison:")
    
    config_results = {}
    
    for config_name, params in processing_configs.items():
        import time
        
        processor = GeometryProcessor(**params)
        
        start_time = time.time()
        config_result = processor.process_route(many_bends_route)
        processing_time = time.time() - start_time
        
        config_results[config_name] = config_result
        
        if config_result.success:
            total_primitives = sum(len(s.primitives) for s in config_result.route.sections)
            total_error = sum(fr.total_error for fr in config_result.fitting_results) if config_result.fitting_results else 0
            
            print(f"  {config_name:12}: {total_primitives:3d} primitives, {total_error:6.2f}mm error, {processing_time:5.2f}s")
        else:
            print(f"  {config_name:12}: Processing failed ({processing_time:5.2f}s)")
else:
    print("Many bends route not available")

## 7. Geometry Validation

Let's examine the geometry validation process and understand common issues.

In [ ]:
# Test geometry validation on edge cases
edge_case_routes = {
    "Very Short": "../tests/data/very_short_route.dxf",
    "Tight Bends": "../tests/data/tight_bend_route.dxf"
}

cable_spec_strict = CableSpec(
    diameter=35.0,
    weight_per_meter=2.5,
    max_tension=8000.0,
    max_sidewall_pressure=500.0,
    min_bend_radius=1500.0,         # Stricter bend radius requirement
)

print("Geometry Validation Results:")

for route_name, route_file in edge_case_routes.items():
    try:
        edge_route = load_route_from_dxf(Path(route_file))
        processor = GeometryProcessor(tolerance_mm=1.0)
        
        # Process with validation
        edge_result = processor.process_route(
            edge_route,
            cable_spec=cable_spec_strict,  # Use strict requirements
            duct_spec=duct_spec
        )
        
        print(f"\n{route_name} Route:")
        print(f"  Processing success: {edge_result.success}")
        
        if edge_result.validation_result:
            validation = edge_result.validation_result
            print(f"  Validation: {'✓' if validation.is_valid else '✗'}")
            print(f"  Issues: {validation.total_errors} errors, {validation.total_warnings} warnings")
            
            # Show specific issues
            for issue in validation.issues[:5]:  # Show first 5 issues
                print(f"    {issue.severity.upper()}: {issue.message}")
                
            if len(validation.issues) > 5:
                print(f"    ... and {len(validation.issues) - 5} more issues")
        
        if edge_result.success and edge_result.route.sections:
            section = edge_result.route.sections[0]
            total_primitives = len(section.primitives)
            print(f"  Fitted primitives: {total_primitives}")
            
            # Check for bend radius violations
            for primitive in section.primitives:
                if hasattr(primitive, 'radius_m'):
                    radius_mm = primitive.radius_m * 1000
                    if radius_mm < cable_spec_strict.min_bend_radius:
                        print(f"    ⚠️  Bend radius {radius_mm:.0f}mm < minimum {cable_spec_strict.min_bend_radius:.0f}mm")
        
    except Exception as e:
        print(f"\n{route_name} Route: Error - {e}")

## 8. Geometry Export and Verification

Let's export fitted geometry and verify the results.

In [ ]:
# Export fitted geometry for verification
from easycablepulling.io.dxf_writer import DXFWriter

# Process a route and export fitted geometry
s_curve = loaded_routes.get("S-Curve")
if s_curve:
    processor = GeometryProcessor(tolerance_mm=1.0)
    fitted_result = processor.process_route(s_curve)
    
    if fitted_result.success:
        # Create DXF writer and export
        writer = DXFWriter()
        
        # Write original route
        writer.write_original_route(s_curve, layer_name="ROUTE_ORIGINAL")
        
        # Write fitted route
        writer.write_fitted_route(fitted_result.route, layer_name="ROUTE_FITTED")
        
        # Write individual primitives
        writer.write_primitives(fitted_result.route)
        
        # Add annotations
        writer.add_length_annotations(fitted_result.route)
        
        # Save to file
        output_path = output_dir / "fitted_geometry.dxf"
        writer.save(output_path)
        
        print(f"✓ Exported fitted geometry: {output_path}")
        
        # Calculate fitting statistics
        if fitted_result.fitting_results:
            fitting_stats = fitted_result.fitting_results[0]
            original_length = s_curve.total_length
            fitted_length = fitted_result.route.total_length
            length_error = abs(fitted_length - original_length)
            length_error_percent = (length_error / original_length) * 100
            
            print(f"\nFitting Statistics:")
            print(f"  Original length: {original_length:.1f}m")
            print(f"  Fitted length: {fitted_length:.1f}m")
            print(f"  Length error: {length_error:.1f}m ({length_error_percent:.2f}%)")
            print(f"  Geometric error: {fitting_stats.total_error:.2f}mm")
            print(f"  Max point error: {fitting_stats.max_error:.2f}mm")
    else:
        print("Fitting failed - cannot export geometry")
else:
    print("S-Curve route not available for export")

## 9. Performance Analysis

Let's analyze the performance characteristics of geometry processing.

In [ ]:
# Performance analysis across different route types
import time

performance_data = []

for route_name, route in loaded_routes.items():
    # Test different tolerance levels
    tolerances = [0.5, 1.0, 2.0, 5.0]
    
    for tolerance in tolerances:
        processor = GeometryProcessor(
            tolerance_mm=tolerance,
            max_iterations=100
        )
        
        # Measure processing time
        start_time = time.time()
        result = processor.process_route(route)
        processing_time = time.time() - start_time
        
        # Collect performance metrics
        if result.success:
            point_count = sum(len(s.original_polyline) for s in route.sections)
            primitive_count = sum(len(s.primitives) for s in result.route.sections)
            total_error = sum(fr.total_error for fr in result.fitting_results) if result.fitting_results else 0
            
            performance_data.append({
                'route_name': route_name,
                'tolerance': tolerance,
                'processing_time': processing_time,
                'point_count': point_count,
                'primitive_count': primitive_count,
                'total_error': total_error,
                'success': True
            })
        else:
            performance_data.append({
                'route_name': route_name,
                'tolerance': tolerance,
                'processing_time': processing_time,
                'point_count': 0,
                'primitive_count': 0,
                'total_error': float('inf'),
                'success': False
            })

# Display performance summary
print("Performance Summary:")
print("Route        Tolerance  Time    Points  Primitives  Error")
print("─" * 60)

for data in performance_data:
    if data['success']:
        print(f"{data['route_name']:12} {data['tolerance']:6.1f}mm  {data['processing_time']:5.2f}s  "
              f"{data['point_count']:6d}  {data['primitive_count']:9d}  {data['total_error']:6.2f}mm")
    else:
        print(f"{data['route_name']:12} {data['tolerance']:6.1f}mm  {data['processing_time']:5.2f}s  FAILED")

In [ ]:
# Visualize performance trends
import pandas as pd

# Convert to DataFrame for easier analysis
df = pd.DataFrame([d for d in performance_data if d['success']])

if not df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Processing time vs tolerance
    for route_name in df['route_name'].unique():
        route_data = df[df['route_name'] == route_name]
        axes[0,0].plot(route_data['tolerance'], route_data['processing_time'], 
                      'o-', label=route_name, linewidth=2, markersize=6)
    
    axes[0,0].set_xlabel('Tolerance (mm)')
    axes[0,0].set_ylabel('Processing Time (s)')
    axes[0,0].set_title('Processing Time vs Tolerance')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].legend()
    
    # Plot 2: Primitives vs tolerance
    for route_name in df['route_name'].unique():
        route_data = df[df['route_name'] == route_name]
        axes[0,1].plot(route_data['tolerance'], route_data['primitive_count'], 
                      's-', label=route_name, linewidth=2, markersize=6)
    
    axes[0,1].set_xlabel('Tolerance (mm)')
    axes[0,1].set_ylabel('Number of Primitives')
    axes[0,1].set_title('Model Complexity vs Tolerance')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].legend()
    
    # Plot 3: Error vs tolerance
    for route_name in df['route_name'].unique():
        route_data = df[df['route_name'] == route_name]
        axes[1,0].semilogy(route_data['tolerance'], route_data['total_error'], 
                          '^-', label=route_name, linewidth=2, markersize=6)
    
    axes[1,0].set_xlabel('Tolerance (mm)')
    axes[1,0].set_ylabel('Total Error (mm)')
    axes[1,0].set_title('Fitting Error vs Tolerance')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].legend()
    
    # Plot 4: Processing time vs complexity
    axes[1,1].scatter(df['point_count'], df['processing_time'], 
                     c=df['tolerance'], s=60, alpha=0.7, cmap='viridis')
    
    cbar = plt.colorbar(axes[1,1].collections[0], ax=axes[1,1])
    cbar.set_label('Tolerance (mm)')
    
    axes[1,1].set_xlabel('Number of Points')
    axes[1,1].set_ylabel('Processing Time (s)')
    axes[1,1].set_title('Processing Time vs Route Complexity')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No performance data to visualize")

## Summary

This advanced tutorial covered:

1. **Geometry fitting algorithms**: Understanding how polylines are converted to mathematical primitives
2. **Fitting accuracy**: Trade-offs between accuracy and model complexity
3. **Route splitting**: Automatic segmentation of long routes
4. **Processing parameters**: Optimizing tolerance and iteration settings
5. **Validation**: Identifying and handling geometry issues
6. **Performance analysis**: Understanding processing time and resource usage
7. **Export capabilities**: Saving fitted geometry for verification

## Key Takeaways

- **Tolerance selection**: Balance between accuracy (low tolerance) and simplicity (high tolerance)
- **Route complexity**: More complex routes require more processing time and careful parameter tuning
- **Validation importance**: Always check validation results for geometry issues
- **Performance scaling**: Processing time scales with route complexity and tolerance requirements
- **Export verification**: Export fitted geometry to verify accuracy in CAD systems

## Best Practices

1. **Start with standard tolerances** (1-2mm) and adjust based on results
2. **Enable splitting** for routes longer than 500m
3. **Check validation results** before proceeding with cable calculations
4. **Export fitted geometry** to verify accuracy against original design
5. **Monitor performance** for batch processing workflows